# AI based resource scaling

## Case scenario

You and your team have been assigned to a project. The business idea is to reduce carbon footprint by saving resources. Your initial investigation shows, that majority of applications that are deployed on Cloud have high and low traffic hours, but the resources are set to support the peak times. That means there are hours when the allocated resources to the application are not utilized.

You want to develop a model which will allow automatic scaling of these resources. For that purpose, your machine learning model should correctly predict the CPU usage.

## Data

The extract of the data for a sample application was provided to you. In the `data` folder you will find 2 files: `train.csv` and `test.csv`. You should conduct your exploration and model building on the `train.csv` file.
Once you are happy with the model, use the `test.csv` file to predict the `cpu_usage`. You should store your predictions in a new csv file which will be called `<your_name>.csv`. The new file should have the following attributes:
- `id` column
- `timestamp` column
- `cpu_usage` column which should hold your predictions

### Data explanation

- `id` - identifier of the record
- `timestamp` - timestamp in the format YYYY-MM-DD HH-MM-SS
- `number_of_requests` - number of requests the application received in the given time
- `number_of_errors` - number of errors that the application logged in the given time
- `response_time` - cummulative time the application took to respond to a request, in miliseconds
- `cpu_cores` - number of CPU cores allocated to the application at a given time (maximum 8 are available)
- `memory_usage` - memory allocation in a given time, in percent
- `cpu_usage` - cpu allocation in a given time, in percent

## Your delivery

At the end of the day, you should provide us with your code in which you demonstrate that you followed these steps:
1. Data load
2. Data cleaning
3. Data Exploration
4. Data Modeling and validation
5. Prediction

The submission should be done as a pull request (PR) on github to this repository. **Please use branch with your name (not main branch)**. The PR should contain the `<your_name>.csv` file stored in the `data` folder and your code stored either in a jupyter notebook or python module.
We have prepared this notebook to help you with the exercise however you are not obliged to use it.

# Evaluation

This exercise does not have one solution. The problem can be approached in multiple ways. In order to evaluate how well your model performs, we will use the  root mean squared error (RMSE) metric. You can learn more about this metric here: https://en.wikipedia.org/wiki/Root-mean-square_deviation

# Get started

## Getting this to your local environment

Make sure you have an account at github.com. We suggest you fork this repository to your own space. Refer here to quickly get the right git commands: https://docs.github.com/en/get-started/quickstart/git-cheatsheet or simply use Github Desktop: https://desktop.github.com/.
Before you start working, make sure that your work can be reproduced later on a different machine.

Hint: define the environment for your project along with all dependencies. Make sure that any random element you use in your code is started from the same seed value.

## Git cheatsheet

If you are familiar with git/github and you know your way around, you can skip this section. 

### Basic Github Flow
Follwing [video](https://ibm.box.com/s/dvym4y5ktbcw8sdv02hecfs5wwe0dn22) describes basic github workflow. It describes how to fork repository, clone it, make some changes, push changes to remote repo and create pull request against original repository. 

### Cloning repo
Before you can clone repo you need either github token or ssh key. If you do not have it setup please follow this [guide](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account)

### Usual workflow
Your usual worklow might look like this:
1. Fork and clone repo on your local (first part of video)
2. Develop your solution, periodically commiting changes when you reach some milestone
3. Push changes to remote server if you are finished or you want just make copy on remote server just in case :-).
4. Once you are done with your solution. Create pull request as show in second part of the video. **Please remember to create pull request against branch with your name (do not use main branch).**

### Useful git commands
Here are some useful git commands:
* ```git clone <repo url>``` - clone repo from remote location to local directory
* ```git add <file|folder>``` - stage your changes 
* ```git commit -m "commit message"``` - commit your changes to local git repo
* ```git push``` - push changes to remote git repo


## 1. Load the data

In [449]:
# Your code goes here
import pandas as pd
import numpy as np

df_test = pd.read_csv('data/test.csv')
df_train = pd.read_csv('data/train.csv')

## 2. Data cleaning

In this step you want to make sure that the data that you work with is loaded correctly, that it does not contain any strange values or that you are not missing any important records. You can read more about this step here: https://en.wikipedia.org/wiki/Data_cleansing

In [450]:
# Your code goes here
df_test = df_test.fillna(0)
df_train = df_train.fillna(0) #cleanig datasets from NaN values

In [451]:
df_test.isnull().values.any()

False

In [452]:
df_train.isnull().values.any() #check for NaN values after cleaning

False

In [453]:
df_test.rename(columns={'Unnamed: 0':'id'}, inplace=True) #add name for missing columns

In [454]:
df_test['id'].is_unique

True

In [455]:
df_train['id'].is_unique

True

In [456]:
drop_c = ['id', 
          'timestamp']
df_test.drop(columns=drop_c, inplace=True, axis=1)
df_train.drop(columns=drop_c, inplace=True, axis=1)

In [457]:
df_train.dtypes.value_counts() 

float64    4
int64      2
dtype: int64

In [458]:
df_test.dtypes.value_counts() 

float64    3
int64      2
dtype: int64

In [459]:
df_test.describe()

,number_of_requests,number_of_errors,response_time,cpu_cores,memory_usage
count,17200.000000,17200.000000,17200.000000,17200.000000,17200.000000
mean,5201.385640,2.491919,12307.822900,3.363488,0.503066
std,10483.977459,22.124663,2037.352316,2.202170,0.289226
min,0.000000,0.000000,0.000000,2.000000,0.000059
25%,249.000000,0.000000,10985.987705,2.000000,0.253181
50%,498.000000,0.000000,11948.646584,2.000000,0.507247
75%,2955.500000,0.000000,13739.689722,4.000000,0.751626
max,49998.000000,690.000000,16773.110102,8.000000,0.999834


In [460]:
df_train.describe()

,number_of_requests,number_of_errors,response_time,cpu_cores,memory_usage,cpu_usage
count,51601.000000,51601.000000,51601.000000,51601.000000,51601.000000,51601.000000
mean,5226.672720,2.644542,12312.780296,3.363036,0.500581,47.676140
std,10550.113172,23.911454,2022.598034,2.202220,0.288363,7.420956
min,0.000000,0.000000,0.000000,2.000000,0.000005,0.000000
25%,252.000000,0.000000,10989.211638,2.000000,0.249912,46.839072
50%,498.000000,0.000000,11943.853696,2.000000,0.502889,47.318397
75%,2948.000000,0.000000,13730.180213,4.000000,0.750690,47.961371
max,49989.000000,875.000000,16817.927832,8.000000,0.999975,100.000000


## 3. Data exploration

This may be one of the most important steps in your analysis. Your objective is to explore patterns in the data that will later drive your decisions about the suitable prediction model. You can read more about this step here: https://en.wikipedia.org/wiki/Data_exploration.

There are many visualization libraries in python which can help you visualize and better understand the relationships between the data. Some of the most used ones are `matplotlib` (https://matplotlib.org/) and `seaborn` (https://seaborn.pydata.org/)

At the end of this step, you should be able to make some important decisions. For example, will you include all features, or only a subset? Will you create new features? How will you treat your target variable? Will you need to encode the data in a different way?

In [465]:
# Your code goes here

has_errors_train = np.where((df_train.number_of_errors >  0),1,df_train.number_of_errors) #encode number_of_errors train
has_errors_test = np.where((df_test.number_of_errors >  0),1,df_test.number_of_errors) #encode number_of_errors test
has_errors_train = {'has_error': has_errors_train}
df_has_errors_train = pd.DataFrame(has_errors_train)
has_errors_test = {'has_error': has_errors_test}
df_has_errors_test = pd.DataFrame(has_errors_test)

In [466]:
df_new_train = df_train.apply(lambda x: x / x.max())
df_new_test = df_test.apply(lambda y: y / y.max())

In [467]:
df_new_train.drop('number_of_errors',inplace=True, axis=1)
df_new_test.drop('number_of_errors',inplace=True, axis=1)

In [468]:
df_new_train = pd.concat([df_has_errors_train, df_new_train], axis=1)
df_new_test = pd.concat([df_has_errors_test, df_new_test], axis=1)

In [469]:
df_new_train

,has_error,number_of_requests,response_time,cpu_cores,memory_usage,cpu_usage
0,0.0,0.195203,0.876610,0.75,0.347608,0.484986
1,1.0,0.199384,0.885793,0.75,0.756432,0.533553
2,1.0,0.104223,0.832988,0.75,0.698485,0.465731
3,1.0,0.147252,0.875048,0.75,0.137353,0.466245
4,0.0,0.193383,0.900745,0.75,0.157937,0.494553
...,...,...,...,...,...,...
51596,1.0,0.186101,0.854805,0.75,0.614510,0.516841
51597,1.0,0.118926,0.854677,0.75,0.209696,0.475718
51598,1.0,0.139211,0.878143,0.75,0.707412,0.475859
51599,0.0,0.165516,0.853385,0.75,0.887705,0.481782


In [470]:
df_new_test

,has_error,number_of_requests,response_time,cpu_cores,memory_usage
0,1.0,0.120265,0.825102,0.75,0.332337
1,0.0,0.176687,0.873875,0.75,0.931371
2,0.0,0.159426,0.842850,0.75,0.803621
3,0.0,0.153066,0.850040,0.75,0.486424
4,0.0,0.146266,0.851457,0.75,0.369106
...,...,...,...,...,...
17195,0.0,0.128025,0.833347,0.75,0.863962
17196,1.0,0.130505,0.858607,0.75,0.300329
17197,1.0,0.166847,0.881303,0.75,0.165297
17198,1.0,0.107104,0.859776,0.75,0.377447


## 4. Data Modeling and Validation

At this point, you should have an idea about your data. That means you are ready for the modeling part. Based on your exploration, you should choose the right type of the model. Since we are operating in a supervised domain (e.g. we know what we want to predict), your main decision should be whether to use a classification, regression or time-series type model.

Scikit-learn (https://scikit-learn.org/stable/index.html) is very broad library for machine learning practitioners. The documentation provides examples for different machine learning problems. Feel free to check it out before you choose your final implementation. If you are not sure which model will work best, you can also train multiple models and choose the best among them.

Do not forget that you need to validate your model. Especially if you train multiple models that you wish to choose from. There are more ways to do the validation (read more here: https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets) but remember that your final solution will be evaluated based on RMSE metric.

In [499]:
# Your code goes here
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

X = df_new_train.loc[:, df_new_train.columns != 'has_error']
y = df_new_train.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

clf = svm.SVC(kernel='rbf')

clf.fit(X_train, y_train) #train model

SVC()

## 5. Prediction

Well done! You are almost finished with the assignment. In this last step you want to use a `predict` method on the data from `test.csv` file. Remember, any transformation or data preprocessing steps you did need to be done on this dataset too.

In [513]:
# Your code goes here
y_pred = clf.predict(X_test)

X1 = df_new_train.loc[:, df_new_train.columns != 'has_error']
y1 = df_new_train.iloc[:, 0]

X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size = 0.2)

y1_pred = clf.predict(X1_test)

print("Accuracy (train):", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))

print("Accuracy (test):", metrics.accuracy_score(y1_test, y_pred))

Accuracy (train): 0.8945838581532797
Precision: 0.7252914343639129
Recall: 0.723823975720789


ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

<div align="center"> Well done! </div>
You have completed all the steps necessary for the assignment. Don't forget to submit your solution according to instructions.
We hope you have enjoyed this and we thank you for your time.